In [1]:
using DifferentialEquations, Plots, Serialization
include("ode_problem.jl")
include("calculate_experimental_outputs.jl")
include("misc.jl");

In [2]:
#figure hyperparameter
marker_size = 7
linewidth = 5
color = :deeppink4;

### This Script Compares GPCR Simulations to the Reported Experimental Measurements

To generate predictions, we set the parameter values to the published values, and simulate the experimental conditions. One condition is the active G Protein response to 1 uM of ligand (Fig5A in Yi et al) from 0 to 600 seconds. The response is stored at 9 timepoints, which recapitulate experimental measurements. The other condition is the dose response of active G Protein from 0.1 to 100 nM of ligand (Fig5A in Yi et al) normalized to the dose at 1 uM (or 1000 nM). Dose responses are stored at 8 concentrations, which again match experimental measurements.

In [3]:
#Load dictionaries containing experimental data
#each dictionary has a key for "save_at", "response", "ligand_stimulation (molecules)", and "average_error"
#dose response also has ligand stimulation amount that all other responses are normalized to
experiment_timecourse = deserialize("outputs/000_processed_active_g_timecourse.dict")
experiment_dose_response = deserialize("outputs/000_processed_active_g_dose_response.dict");
test_dose_response = deserialize("outputs/000_processed_rl_dose_response.dict");

Timecourse Figure 5A

In [4]:
#create ODEProblem
ligand_dose = 1000.0
odesys, u0, tspan, p = return_ode_problem_default_inputs(ligand_dose)
op = DifferentialEquations.ODEProblem(odesys, u0, tspan, p)
solver_inputs = return_ode_problem_solver_default_inputs("timecourse")

#simulate
predicted = DifferentialEquations.solve(op, solver_inputs["solver"], abstol=solver_inputs["abstol"], reltol=solver_inputs["reltol"], saveat=solver_inputs["saveat"]);
fraction_activated = calculate_all_quantities(predicted)["fraction_active_G"]

#fine grain simulation for visualization
save_at = return_finegrain_saveat()
predicted = DifferentialEquations.solve(op, solver_inputs["solver"], abstol=solver_inputs["abstol"], reltol=solver_inputs["reltol"], saveat=save_at);
fraction_activated_finegrain = calculate_all_quantities(predicted)["fraction_active_G"];

#plot
p1 = plot(save_at, fraction_activated_finegrain, label=false, color=color, linewidth=linewidth)
scatter!(experiment_timecourse["save_at"], fraction_activated, label="simulated", color=color, markersize=marker_size)
scatter!(experiment_timecourse["save_at"], experiment_timecourse["response"], label="experimental data", color="black",
yerr=experiment_timecourse["average_error"], markersize=marker_size)
xlabel!("Time (secs)")
ylabel!("Active G-Proteins (Fraction)")
#title!("Figure 5A of Yi et al")
ylims!(0,0.5);
plot!(size=(690,690), dpi=300)
savefig("outputs/100_yi_figure5a.png")

"/Users/hollyhuber/Documents/structure_informed_cell_signaling2/gpcr/outputs/100_yi_figure5a.png"

Dose Response Figure 5B and 4A

In [6]:
#ode problems
normalize_to = normalize_to_dose()
ligand_doses = cat(return_ligand_dose_nM(),normalize_to, dims=1) #normalize to last stimulation

n_doses = length(ligand_doses)
#next input we need is an ode problem
odeproblems = []
for l in ligand_doses
    odesys, u0, tspan, p = return_ode_problem_default_inputs(l)
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p)
    push!(odeproblems, prob)
end

#solver inputs 
solver_inputs = return_ode_problem_solver_default_inputs("dose response");

active_g = []
rl = []
for i in 1:n_doses
    predicted = DifferentialEquations.solve(odeproblems[i], solver_inputs["solver"], abstol=solver_inputs["abstol"], reltol=solver_inputs["reltol"], saveat=solver_inputs["saveat"])
    fraction_activated = calculate_all_quantities(predicted)
    push!(active_g, fraction_activated["fraction_active_G"][1]) #need to index vector of 1d
    push!(rl, fraction_activated["RL"][1])
end

active_g = active_g./active_g[end]
rl = rl./rl[end]

#finegrain dose response
ligand_doses = return_finegrain_ligand_dose_nM() #(nM) 
n_doses = length(ligand_doses)
#next input we need is an ode problem
odeproblems = []
L_index = species_indices_map()["L"]
for l in ligand_doses #replace ligand concentration in u0
    odesys, u0, tspan, p = return_ode_problem_default_inputs(l) #get p, tspan, odesys
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p)
    push!(odeproblems, prob)
end

active_g_finegrain = []
rl_finegrain = []

for i in 1:n_doses
    predicted = DifferentialEquations.solve(odeproblems[i], solver_inputs["solver"], abstol=solver_inputs["abstol"], reltol=solver_inputs["reltol"], saveat=solver_inputs["saveat"])
    fraction_activated = calculate_all_quantities(predicted)
    push!(active_g_finegrain, fraction_activated["fraction_active_G"][1])
    push!(rl_finegrain, fraction_activated["RL"][1])
end

active_g_finegrain = active_g_finegrain./active_g_finegrain[end]
rl_finegrain = rl_finegrain./rl_finegrain[end]

#plot active g
using CSV, DataFrames
dose_response_exp_ligand_nM = log10.(return_ligand_dose_nM())
p2 = plot(collect(range(start=-2,stop=3,length=50)) , active_g_finegrain, label=false, color=color, linewidth=linewidth)
scatter!(dose_response_exp_ligand_nM, active_g, label=false, color=color, markersize=marker_size)
scatter!(dose_response_exp_ligand_nM, experiment_dose_response["response"], label=false, color="black",yerr=experiment_dose_response["average_error"], markersize=marker_size)
xlabel!("log[Alpha-Factor](nM)")
ylabel!("G-Protein Activation Response")
xlims!(-3,3)
ylims!(0,1.2)
#title!("Figure 5B of Yi et al");
plot!(size=(690,690), dpi=300)
savefig("outputs/100_yi_figure5b.png");

#plot rl
using CSV, DataFrames
dose_response_exp_ligand_nM = log10.(return_ligand_dose_nM())
p2 = plot(collect(range(start=-2,stop=3,length=50)) , rl_finegrain, label=false, color=color, linewidth=linewidth)
scatter!(dose_response_exp_ligand_nM, rl, label=false, color=color, markersize=marker_size)
scatter!(dose_response_exp_ligand_nM, test_dose_response["response"], label=false, color="black",yerr=test_dose_response["average_error"], markersize=marker_size)
xlabel!("log[Alpha-Factor](nM)")
ylabel!("RL Response")
xlims!(-3,3)
ylims!(0,1.2)
#title!("Figure 5B of Yi et al");
plot!(size=(690,690), dpi=300)
savefig("outputs/100_yi_figure4a.png");